In [1]:
"""
You need to run this cell for the code in following cells to work.
"""

# Enable module reloading
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

import datetime
import os
import sys
sys.path.append('..')

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from week_4.backstage.load_data import load_data
from week_4.backstage.utils import *

# Week 4

__Goals for this week__

We will start working with _TensorFlow_ - a modern deep learning framework. We will implement a multilayer perceptron model and train it with this framework.

__Feedback__

This lab is a work in progress. If you notice a mistake, notify us or you can even make a pull request. Also please fill the [questionnaire](https://forms.gle/r27nBAvnMC7jbjJ58) after you finish this lab to give us feedback.

## Reminder

__You submit your project proposal in a week.__

## TensorFlow

TensorFlow (TF) is a state-of-the-art framework for neural network development, training and deployment. It provides:

1. Basic building blocks for models - layers, loss functions, activation functions, etc. The neural models are mostly built from common building blocks.
2. Auto-differentiation. We do not have to calculate the derivatives of the loss function w.r.t. parameters. Instead these quantities are derived automatically. The training algorithms, such as SGD, can also be used via handy API.
3. Toolset for visualization, deployment, distributed computing, etc.

We will use _TensorFlow 2.0_ in our labs.

### Tensors

`Tensor` is the basic TF type. Constants called by `tf.constant` are immutable, while variables called by `tf.Variable` can be changed later. Therefore variables are used as model parameters. Notice that each tensor has a `shape` and a `dtype`:

In [2]:
np_array = np.arange(15, dtype=np.float).reshape(5,3)
print(np_array)
print()

tf_tensor = tf.constant(np_array)
print(tf_tensor)


We can do the usual matrix operations with these tensors, e.g. $\sigma(\mathbf{Wx} + \mathbf{b})$. Common operators, such as `+` for addition or `@` for matrix multiplication are supported:

In [3]:
def sigma(x):
    return 1 / (1 + tf.exp(-x))

w = tf.constant([
    [0.1, 0.2, 0.3],
    [0.2, 0.1, 0.3],
    [0.3, 0.1, 0.2]
])

x = tf.constant([
    [0.5],
    [-0.3],
    [0.2],
])

b = tf.constant([
    [0.3],
    [-0.4],
    [0.2],
])

print(sigma(w @ x + b))


Note, that we need to define $\mathbf{x}$ and $\mathbf{b}$ as 2D tensors with shape `(3, 1)` if we want them to behave like column vectors.

### Defining models

Instead of working with these tensors, we will use a high-level API for model definition called `keras`. Within this API we have `tf.keras.Model` class for models. `keras` models are basic computational units that transform input $x$ to output $\hat{y}$ and that can be trained via SGD or similar algorithms. 

We will define it using predefined `Layer`s. Compared to `keras` models, layers are more atomic computational units, that can be reused, e.g. `Dense` layer is an implementation of MLP layer equation: $\sigma(\mathbf{Wx} + \mathbf{b})$. Carefully read the following code:

In [4]:
class MultilayerPerceptron(keras.Model):  # Subclassing
    
    def __init__(self, dim_output, dim_hidden, num_layers=1, activation=keras.activations.linear):
        super(MultilayerPerceptron, self).__init__(name='multilayer_perceptron')
        self.dim_output = dim_output
        self.dim_hidden = dim_hidden

        # Within Model.__init__ we initialize all the layers we will use
        self.hidden_layers = []
        for _ in range(num_layers):
            layer = keras.layers.Dense(units=dim_hidden, activation=activation)
            self.hidden_layers.append(layer)
        self.layer_o = keras.layers.Dense(units=dim_output, activation=keras.activations.softmax)

    def call(self, x):  # call defines the flow of the computation, e.g. in this particular model
                        # we simply call the two layers one after the oter
        h = x
        for layer in self.hidden_layers:
            h = layer(h)
        y = self.layer_o(h)
        return y


__Exercise 4.1:__ Check the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) of `Dense` layer. The arguments of the layer provide us with additional options, you should know at least the first five arguments. Looking at thes arguments, what is missing in the definition used above?

### Training models

We will train this model to classify the _Iris_ dataset from previous lab. Training models defined like this is really easy:

In [ ]:
data = load_data('iris.csv', num_classes=3)

# Reminder how these data look
for x, y in list(zip(data.x, data.y))[:5]:  # First 5 samples
    print(x, y)

In [6]:
model = MultilayerPerceptron(  # We create a new model
    dim_output=3,
    dim_hidden=32)

model.compile(  # By compiling we prepare the model for training
    optimizer=keras.optimizers.SGD(learning_rate=0.003),  # We pick a optimizer algorithm
    loss='mean_squared_error',  # We pick a loss function
    metrics=['accuracy'])  # We pick evaluation metrics

model.fit(  # Fit runs the training over provided data
    x=data.x,
    y=data.y,
    batch_size=4,
    epochs=20)


Train on 150 samples
Epoch 1/20
150/150 [==============================] - 0s 3ms/sample - loss: 0.4304 - accuracy: 0.3333
Epoch 2/20
150/150 [==============================] - 0s 354us/sample - loss: 0.4238 - accuracy: 0.3333
Epoch 3/20
150/150 [==============================] - 0s 397us/sample - loss: 0.4160 - accuracy: 0.3333
Epoch 4/20
150/150 [==============================] - 0s 338us/sample - loss: 0.4105 - accuracy: 0.3333
Epoch 5/20
150/150 [==============================] - 0s 340us/sample - loss: 0.4056 - accuracy: 0.3333
Epoch 6/20
150/150 [==============================] - 0s 348us/sample - loss: 0.3981 - accuracy: 0.3200
Epoch 7/20
150/150 [==============================] - 0s 391us/sample - loss: 0.3844 - accuracy: 0.3200
Epoch 8/20
150/150 [==============================] - 0s 333us/sample - loss: 0.3597 - accuracy: 0.2933
Epoch 9/20
150/150 [==============================] - 0s 330us/sample - loss: 0.3326 - accuracy: 0.1267
Epoch 10/20
150/150 [========================

This is the selling point for using modern neural frameworks. The model is trained via SGD, but we do not need to calculate derivatives. Instead they are calculated automatically by TF. We also do not need to program how SGD works, nor we need to define the loss functions or metrics.

All that we done manually last week is now hidden behind the `fit` function. You should already be familiar with all the concepts that were introduced in the code above, such as `epochs`, `batch_size`, `metrics`, `loss`, `optimizer`, etc.

### Programming assignment 4.2: Multilayer Perceptron [1pt]

Extend the `MultilayerPerceptron` definition above so that we can have model:

1. _Arbitrary number of layers:_ `num_layers=1` tells us that we have one hidden layer in our model.
2. _Arbitrary activation functions:_ Function is passed as an argument in this case.

Check the first command below to see how is the new object created. You can play around with various hyperparameters to see how do the results change.

In [7]:
model = MultilayerPerceptron(
    dim_output=3,
    dim_hidden=32,
    num_layers=3,
    activation=keras.activations.sigmoid)

# compile and fit are the same as above
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01),
    loss='mean_squared_error',
    metrics=['accuracy'])

model.fit(
    x=data.x,
    y=data.y,
    batch_size=4,
    epochs=20)


Train on 150 samples
Epoch 1/20
150/150 [==============================] - 0s 3ms/sample - loss: 0.2663 - accuracy: 0.3333
Epoch 2/20
150/150 [==============================] - 0s 426us/sample - loss: 0.2544 - accuracy: 0.3333
Epoch 3/20
150/150 [==============================] - 0s 424us/sample - loss: 0.2484 - accuracy: 0.3333
Epoch 4/20
150/150 [==============================] - 0s 435us/sample - loss: 0.2444 - accuracy: 0.3333
Epoch 5/20
150/150 [==============================] - 0s 355us/sample - loss: 0.2409 - accuracy: 0.3200
Epoch 6/20
150/150 [==============================] - 0s 435us/sample - loss: 0.2377 - accuracy: 0.3333
Epoch 7/20
150/150 [==============================] - 0s 413us/sample - loss: 0.2341 - accuracy: 0.3467
Epoch 8/20
150/150 [==============================] - 0s 383us/sample - loss: 0.2314 - accuracy: 0.3400
Epoch 9/20
150/150 [==============================] - 0s 389us/sample - loss: 0.2291 - accuracy: 0.3467
Epoch 10/20
150/150 [========================

### Building blocks

Apart from automatic training, TF also provides us with a lot of pre-programmed parts, such as:

- [Loss functions](https://www.tensorflow.org/api_docs/python/tf/keras/losses), such as mean squared error.
- [Activation functions](https://www.tensorflow.org/api_docs/python/tf/keras/activations), such as sigmoid or ReLU.
- [Optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers), such as SGD.
- [Metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics), such as accuracy.
- [Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers), such as Dense layer, which is basically the MLP layer $\sigma(\mathbf{Wx} + \mathbf{b})$.
- [Initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers), such as Glorot (Xavier) initialization.
- and many other features.

You have seen an example of each of these parts in the code above. E.g. we can use `loss='mean_squared_error'` because a loss function with such name is defined in `keras.losses`. You should be able to program most of your projects with these pre-programmed building blocks. But you can of course define your own blocks by following the documentation.

### Evaluation

`fit` also has a support for creating a test set, called `validation set` in TF. By using `validation_split` in `fit` it uses part of the data for evaluation. This is the same concept we practiced last week. Check the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) of `fit` to see what other options it has - you should understand most of them already. Run the following code to see how it looks like:


In [8]:
model = MultilayerPerceptron(
    dim_output=3,
    dim_hidden=32,
    num_layers=3,
    activation=keras.activations.sigmoid)

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.003),
    loss='mean_squared_error',
    metrics=['accuracy'])

model.fit(
    x=data.x,
    y=data.y,
    batch_size=4,
    epochs=20,
    validation_split=0.2)  # This was added


Train on 120 samples, validate on 30 samples
Epoch 1/20
120/120 [==============================] - 0s 3ms/sample - loss: 0.2453 - accuracy: 0.3417 - val_loss: 0.2478 - val_accuracy: 0.3000
Epoch 2/20
120/120 [==============================] - 0s 608us/sample - loss: 0.2439 - accuracy: 0.3417 - val_loss: 0.2460 - val_accuracy: 0.3000
Epoch 3/20
120/120 [==============================] - 0s 561us/sample - loss: 0.2426 - accuracy: 0.3417 - val_loss: 0.2445 - val_accuracy: 0.3000
Epoch 4/20
120/120 [==============================] - 0s 510us/sample - loss: 0.2414 - accuracy: 0.3417 - val_loss: 0.2430 - val_accuracy: 0.3000
Epoch 5/20
120/120 [==============================] - 0s 522us/sample - loss: 0.2403 - accuracy: 0.3417 - val_loss: 0.2417 - val_accuracy: 0.3000
Epoch 6/20
120/120 [==============================] - 0s 491us/sample - loss: 0.2393 - accuracy: 0.3417 - val_loss: 0.2405 - val_accuracy: 0.3000
Epoch 7/20
120/120 [==============================] - 0s 554us/sample - loss: 0.2

## TensorBoard

_TensorBoard_ (TB) is a great visualization tool for training TF models. First, we need to tell the model that it should create TB-related logs during training. The easiest way is to use callbacks: 

In [21]:
model = MultilayerPerceptron(
    dim_output=3,
    dim_hidden=32,
    num_layers=3,
    activation=keras.activations.sigmoid)

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.003),
    loss='mean_squared_error',
    metrics=['accuracy'])

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=os.path.join("logs", timestamp()),
    histogram_freq=1)

model.fit(
    x=data.x,
    y=data.y,
    batch_size=4,
    epochs=20,
    validation_split=0.2,
    callbacks=[tensorboard_callback],  # Callback
    verbose=0)  # Supressing text output

Now we can visulize the results with TensorBoard. We can start it from terminal, or directly from jupyter (run the next cell). After you start TB, you can access it from the notebook, or more conveniently directly from your browser at http://localhost:6006.

The first two tabs _Scalars_ and _Graphs_ are the most interesting to you right now. The first shows how do various quantities change during the training. It shows them for both train and validation data. You can also see the results for multiple runs at the same time. Run the training above again, but change some hyperparameters, e.g. learning rate. Then you can directly compare the results in TB.

_Graphs_ show a graph of your model, i.e. how does it compute the results. By double-clicking you can open individual parts and see how are they defined, i.e. open your model and then a dense layer within to see how is it defined.

In [22]:
%tensorboard --logdir logs --bind_all

Reusing TensorBoard on port 6006 (pid 61), started 1:05:10 ago. (Use '!kill 61' to kill it.)

### Programming assignment 4.3: Multilayer Perceptron, Part 2 [1pt]

First, run the current model with TB callback to get some results. Then implement the following changes:

1. Use _softmax_ as an activation function, but for the __last__ layer. Use the activation provided by `activation` argument for all the other - hidden - layers.
2. Use _categorical crossentropy_ as a loss function.

You can find both these functions in the links of pre-programmed building blocks above. Then compare the results of this new implementation in TensorBoard.

#### Submission

Save the code for your `MultilayerPerceptron` and you training commands (e.g. `compile`, `fit`, etc.) in a `mlp.py` file and submit them to AIS. You need to complete __PA 4.2__ before proceeding to __PA 4.3__. There are no tests this week so consult the submission with your teacher if needed.

## Gradient Tape

`fit` is a very convenient way of training neural models, but sometimes we need more flexibility and control. For example, with `fit` we can not track the training step by step (e.g. for debugging). The model is compiled into a computation graph in the background. So if you want to have a debugging print within a model, it will not run. E.g., try printing the value of `h` in the model `call`.

Instead we can use so called `GradientType`. With this tape the debugging print of `h` will run. Check the following code, it is very similar in how we defined SGD in previous labs:

In [20]:
model = MultilayerPerceptron(
    dim_output=3,
    dim_hidden=32)

optimizer = keras.optimizers.SGD(learning_rate=0.01)
loss_function = keras.losses.MeanSquaredError()

# loss_function = keras.losses.CategoricalCrossentropy()
# You can use cross-entropy loss if you completed PA 4.3
    
def step(xs, ys):  # This has the same meaning as step function in previous labs
    
    with tf.GradientTape() as tape:
        preds = model(xs)  # Model predictions
        loss = loss_function(ys, preds)  # The value of loss function comparing the true
                                         # values ys with predictions

    gradient = tape.gradient(
        target=loss,
        sources=model.trainable_variables)  # Calculate the gradient of loss function w.r.t. model parameters.
                                            # This behaves the same as gradient methods from previous labs.
        
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))  # Applies the computed gradient on current
                                                                         # parameter values.
    
def loss(xs, ys):
    preds = model(xs)
    return loss_function(ys, preds)
    
num_epochs = 100
batch_size = 5
num_samples = len(data.x)

# Training loop (without shuffling for simplicity)
for e in range(num_epochs):
    for i in np.arange(0, num_samples, batch_size):  # Batching
        step(data.x[i:i+batch_size], data.y[i:i+batch_size])
    print('Epoch:', e, 'Loss:', loss(data.x, data.y).numpy())
        


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 0 Loss: 0.3324832
Epoch: 1 Loss: 0.2617142
Epoch: 2 Loss: 0.16195057
Epoch: 3 Loss: 0.13432351
Epoch: 4 Loss: 0.12554379
Epoch: 5 Loss: 0.119047165
Epoch: 6 Loss: 0.114001796
Epoch: 7 Loss: 0.10989445
Epoch: 8 Loss: 0.10641589
Epoch: 9 Loss: 0.10337446
Epoch: 10 Loss: 0.100647815
Epoch: 11 Loss: 0.09815552
Epoch: 12 Loss: 0.09584325
Epoch: 13 Loss: 0.09367351
Epoch: 14 Loss: 0.091619775
Epoch: 15 Loss: 0.08966307
Epoch: 16 Loss: 0.0877894
Epoch: 17 Loss: 0.08598843
Epoch: 18 Loss: 0.08425234
Epoc

## Further Reading

Check out TF [Tutorials](https://www.tensorflow.org/tutorials) and [Guide](https://www.tensorflow.org/guide) for some further reading. Note that all the documents there are in fact Jupyter notebooks, so you can download them and run them here. At this point, you should check:

- _Tutorials > ML basics with Keras_ - for some basic practical use cases.
- _Tutorials > Load and preproccess data_ - to see how can you load data using TF.

You can also check:

- _Guide > Keras_ - for more in-depth explanation of how TF works.

This [notebook](https://colab.research.google.com/drive/1UCJt8EYjlzCs1H1d1X0iDGYJsHKwu-NO) is also a nice showcase of additional TF features.

## Correct Answers

__E 4.1:__ Activation function is missing. `Dense` uses linear activation by default. We need to use the `activation` argument to add an activation function. There we can use our own activation functions, such as `sigma` defined previously. Or, we can use some of the pre-programmed activation functions from `tf.keras.activations` module.